In [1]:
import pygame
import os
import time 
import random
pygame.font.init()


WIDTH, HEIGHT = 750, 750
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Space Invader")

path = 'C:\\Users\\BaDWoLF4096\\Desktop\\Space Invader'

#loaded ships
RED_SPACE_SHIP = pygame.image.load(os.path.join(path,'assets','pixel_ship_red_small.png'))
GREEN_SPACE_SHIP = pygame.image.load(os.path.join(path,'assets','pixel_ship_green_small.png'))
BLUE_SPACE_SHIP = pygame.image.load(os.path.join(path,'assets','pixel_ship_blue_small.png'))

#player ship
YELLOW_SPACE_SHIP = pygame.image.load(os.path.join(path,'assets','pixel_ship_yellow.png'))



#lasers
RED_LASER = pygame.image.load(os.path.join(path,'assets','pixel_laser_red.png'))
GREEN_LASER = pygame.image.load(os.path.join(path,'assets','pixel_laser_green.png'))
BLUE_LASER = pygame.image.load(os.path.join(path,'assets','pixel_laser_blue.png'))
YELLOW_LASER = pygame.image.load(os.path.join(path,'assets','pixel_laser_yellow.png'))



#background
BG = pygame.transform.scale(pygame.image.load(os.path.join(path,'assets','background-black.png')),(WIDTH, HEIGHT))


class Laser:
    def __init__(self, x, y, img):
        self.x = x
        self.y = y
        self.img = img
        self.mask = pygame.mask.from_surface(self.img)
        
    def draw(self, window):
        window.blit(self.img, (self.x, self.y))
        
    def move(self, vel):
        self.y += vel
        
    def off_screen(self, height):
        return not (self.y <= height and self.y >= 0)
    
    def collision(self, obj):
        return collide(obj, self)
    
    

class Ship:
    COOLDOWN = 30
    
    def __init__(self, x, y, health=100):
        self.x = x
        self.y = y
        self.health = health
        self.ship_img = None
        self.laser_img = None
        self.laser = []
        self.cool_down_counter = 0

    def draw(self, window):
        WIN.blit(self.ship_img, (self.x, self.y))
        for laser in self.laser:
            laser.draw(window)
    
    def move_laser(self,  vel, obj):
        self.cooldown()
        for laser in self.laser:
            laser.move(vel)
            if laser.off_screen(HEIGHT):
                self.laser.remove(laser)
            elif laser.collision(obj):
                obj.health -= 10
                self.laser.remove(laser)

    
    def cooldown(self):
        if self.cool_down_counter >= self.COOLDOWN:
            self.cool_down_counter = 0
        elif self.cool_down_counter > 0:
            self.cool_down_counter += 1
        
    def shoot(self):
        if self.cool_down_counter == 0:
            laser = Laser(self.x, self.y, self.laser_img)
            self.laser.append(laser)
            self.cool_down_counter = 1
            
    def get_width(self):
        return self.ship_img.get_width()
    
    def get_height(self):
        return self.ship_img.get_height()
        
class Player(Ship):
    def __init__(self, x, y, health=100):
        super().__init__(x, y, health)
        self.ship_img = YELLOW_SPACE_SHIP
        self.laser_img = YELLOW_LASER
        self.mask = pygame.mask.from_surface(self.ship_img)
        self.max_health = health
        
        
        
        
    def move_laser(self, vel, objs):
        self.cooldown()
        for laser in self.laser:
            laser.move(vel)
            if laser.off_screen(HEIGHT):
                self.laser.remove(laser)
            else:
                for obj in objs:
                    if laser.collision(obj):
                        objs.remove(obj)
                        if laser in self.laser:
                            self.laser.remove(laser)

        
    def collision(self, obj):
        return collide(obj, self)
        
class Enemy(Ship):
    COLOR_MAP = {
        "red": (RED_SPACE_SHIP, RED_LASER),
        "blue": (BLUE_SPACE_SHIP, BLUE_LASER),
        "green": (GREEN_SPACE_SHIP, GREEN_LASER)
    }
    def __init__(self, x, y, color, health=100):
        super().__init__(x, y, health)
        self.ship_img, self.laser_img = self.COLOR_MAP[color]
        self.mask = pygame.mask.from_surface(self.ship_img)
        
    def move(self, vel):
        self.y += vel
        
    def shoot(self):
        if self.cool_down_counter == 0:
            laser = Laser(self.x - 19, self.y, self.laser_img)
            self.laser.append(laser)
            self.cool_down_counter = 1
        
        
def collide(obj1, obj2):
    offset_x = obj2.x - obj1.x
    offset_y = obj2.y - obj1.y
    return obj1.mask.overlap(obj2.mask, (offset_x, offset_y)) !=  None
        
def main():
    run = True
    FPS = 60
    level = 0
    lives = 5
    main_font = pygame.font.SysFont("comicsans", 50)
    clock = pygame.time.Clock()
    
    enemies = []
    wave_length = 5
    enemy_val = 1
    
    laser_vel = 4
    player_val = 5
    
    
    player = Player(300, 640)
    
    def redraw_window():
        WIN.blit(BG, (0,0))
        lives_label = main_font.render(f"Lives:{lives}", 1, (255,255,255))
        level_label = main_font.render(f"Level:{level}", 1, (255,255,255))
        lost = main_font.render
        
        WIN.blit(lives_label, (10,10))
        WIN.blit(level_label, (WIDTH - level_label.get_width()-10 , 10))
        
        for enemy in enemies:
            enemy.draw(WIN)
            
            
        player.draw(WIN)
        
        
        
        pygame.display.update()
        
    
    while run:
        clock.tick(FPS)
        redraw_window()
        
        if lives <= 0 or player.health <= 0:
            print('lose')
            run = False
        
        if len(enemies) == 0:
            level += 1
            wave_length += 5
            
            for i in range(wave_length):
                enemy = Enemy(random.randrange(50, WIDTH-100) ,random.randrange(-1500, -100) ,random.choice(["red","blue","green"]))
                enemies.append(enemy)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quit()

        keys = pygame.key.get_pressed()
        if keys[pygame.K_a] and player.x - player_val > 0:
            player.x -= player_val
        if keys[pygame.K_d] and player.x + player_val + player.get_width() - 10  < WIDTH:
            player.x += player_val
        if keys[pygame.K_w] and player.y - player_val > 0:
            player.y -= player_val
        if keys[pygame.K_s] and player.y + player_val + player.get_height() + 10 < HEIGHT:
            player.y += player_val 
        if keys[pygame.K_SPACE]:
            player.shoot()
        
        
        for enemy in enemies[:]:
            enemy.move(enemy_val)
            enemy.move_laser(laser_vel, player)
            
            if random.randrange(0, 2*60) == 1:
                enemy.shoot()
            
            if enemy.y + enemy.get_height() > HEIGHT:
                lives -= 1
                enemies.remove(enemy)
                
            if player.collision(enemy):
                player.health -= 10
                enemies.remove(enemy)
        
        player.move_laser(-laser_vel, enemies)
        
        

main()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
lose
